# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
print(len(df))
df.head()

8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [3]:
# 1. Count the number of viewers who also clicked
viewers_clicked = df[(df['action'] == 'view') & (df['id'].isin(df[df['action'] == 'click']['id']))]
print("Number of viewers who also clicked:", len(viewers_clicked))

Number of viewers who also clicked: 1860


In [4]:
# 2. Count the number of viewers who didn't click
viewers_not_clicked = df[(df['action'] == 'view') & (~df['id'].isin(df[df['action'] == 'click']['id']))]
print("Number of viewers who didn't click:", len(viewers_not_clicked))


Number of viewers who didn't click: 4468


In [5]:
# 3. Check for anomalies
# - Verify if anyone clicked who didn't view (anomalous behavior)
clickers_not_viewed = df[(df['action'] == 'click') & (~df['id'].isin(df[df['action'] == 'view']['id']))]
print("Number of clickers who didn't view:", len(clickers_not_viewed))


Number of clickers who didn't view: 0


In [7]:
# 4. Examine overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id']
experiment_group = df[df['group'] == 'experiment']['id']

overlap = control_group[control_group.isin(experiment_group)]
print("Number of users in both control and experiment groups:", len(overlap))

Number of users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [23]:
#Your code here
import numpy as np
import scipy.stats as stats

df['count'] = 1

# Create subsets for the control and experiment groups
control_data = df[df['group'] == 'control'].pivot(index='id', columns='action', values='count')
control_data = control_data.fillna(value=0)

experiment_data = df[df['group'] == 'experiment'].pivot(index='id', columns='action', values='count')
experiment_data = experiment_data.fillna(value=0)

print(control_data)
print(experiment_data)

action  click  view
id                 
182994    1.0   1.0
183089    0.0   1.0
183248    1.0   1.0
183515    0.0   1.0
183524    0.0   1.0
...       ...   ...
936786    0.0   1.0
937003    0.0   1.0
937073    0.0   1.0
937108    0.0   1.0
937217    1.0   1.0

[3332 rows x 2 columns]
action  click  view
id                 
182988    0.0   1.0
183136    0.0   1.0
183141    1.0   1.0
183283    0.0   1.0
183389    0.0   1.0
...       ...   ...
935382    0.0   1.0
935576    0.0   1.0
935742    1.0   1.0
936129    0.0   1.0
937139    1.0   1.0

[2996 rows x 2 columns]


In [24]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """
   # Calculate the difference in means between the two groups
    numerator = a.mean() - b.mean()

    # Calculate the denominator, which is the standard error
    # The standard error takes into account the variances of the two groups
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)

    # Return the absolute value of the t statistic
    return np.abs(numerator/denominator)

In [25]:
def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    # Calculate the sample variances and sizes for both groups
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size

     # Calculate the numerator and denominator for the degrees of freedom
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
     # Return the degrees of freedom for Welch's t-test
    return numerator/denominator

In [26]:
def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """

    # Calculate the t statistic using the Welch's t function
    t = welch_t(a, b)

    # Calculate the degrees of freedom using the Welch's degrees of freedom function
    df = welch_df(a, b)

    # Calculate the p-value using the cumulative distribution function (CDF) of the t-distribution
    p = 1-stats.t.cdf(np.abs(t), df)

    # If a two-sided test is requested, double the p-value to account for both tails of the distribution
    
    if two_sided:
        return 2*p
    else:
        return p


In [27]:
p_value_welch_ttest(control_data.click, experiment_data.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [28]:
#Your code here
control_rate = control_data.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment_data)
expected_experiment_clicks_under_null

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [31]:
#Your code here
# Calculate the standard error
n = len(experiment_data)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(f'Standard deviation: {std}')
# Calculate the z-score
actual_experiment_clicks = experiment_data.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(f'z-score : {z_score}')



Standard deviation: 24.568547907005815
z-score : 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [32]:
#Your code here
import scipy.stats as stats
# Calculate the p-value
p_value = 1 - stats.norm.cdf(z_score) 
p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [33]:
"""
Yes, it seems that the experimental page design is more effective in comparison to the control page design.
Both results provide strong evidence to reject the null hypothesis,even though the p-value is only slightly lower.
"""

'\nYes, it seems that the experimental page design is more effective in comparison to the control page design.\nBoth results provide strong evidence to reject the null hypothesis,even though the p-value is only slightly lower.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.